In [1]:
from keras.models import load_model
from PIL import Image
from numpy import expand_dims
from numpy import asarray

import numpy as np
import pickle
import cv2

In [2]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = load_model("../Models/FaceNet/facenet_keras.h5")

myfile = open("../Data/signature.pkl", "rb")
db = pickle.load(myfile)
myfile.close()

In [3]:
db

{'yasir': array([[ 4.7496393e-01, -5.1433891e-02, -1.7465513e+00,  1.3597366e-01,
         -2.3277947e-01,  1.8568149e+00, -2.7333647e-02, -6.8764997e-01,
         -7.3805398e-01,  8.7168950e-01, -5.1251209e-01, -1.1306393e+00,
          1.8104613e-03, -1.9447037e+00, -1.0142580e-01, -2.3805945e+00,
          2.0721355e-01, -7.1974701e-01,  5.2551079e-01, -1.7409155e+00,
          7.4455237e-01, -8.7175381e-01, -1.2435668e-01, -8.5003263e-01,
          2.9337516e-01,  6.5471089e-01, -4.5002532e-01,  4.6997073e-01,
         -3.7316722e-01, -1.0898532e-01,  8.6771595e-01,  5.9517372e-01,
          1.3337758e+00, -2.0152993e+00,  1.0477431e+00, -1.2800945e+00,
          2.8449321e-01, -2.5943071e-01,  8.6950248e-01, -6.4500356e-01,
          3.2720223e-01,  4.7772032e-01, -5.7759070e-01, -1.8188520e-01,
         -1.6262227e-01, -3.5108480e-01, -4.3932545e-01,  2.2523146e+00,
         -6.1125970e-01,  6.2886322e-01, -7.0966363e-02,  1.2625568e-04,
          2.5461742e-01,  6.5137750e-01,  

In [4]:
cap = cv2.VideoCapture(1)

while(1):
    _, imgVideo = cap.read()
    
    FaceDetect = HaarCascade.detectMultiScale(imgVideo, 1.3, 10)
    
    for (x1, y1, width, height) in FaceDetect:
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        
        img = cv2.cvtColor(imgVideo, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_array = asarray(img)
        
        face = img_array[y1:y2, x1:x2]
        
        face = Image.fromarray(face)
        face = face.resize((160,160))
        face = asarray(face)
        
        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        
        face = expand_dims(face, axis=0)
        signature = MyFaceNet.predict(face)
        # print(signature)
        
        min_dist = 100
        identify = ''
        for key, value in db.items():
            dist = np.linalg.norm(value-signature)
            # print(dist)
            if (dist < min_dist):
                min_dist = dist 
                if min_dist > 7:
                    identify = "Unknown"
                else :
                    identify  = key
                    print(identify,",",min_dist)
                
        cv2.rectangle(imgVideo, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.rectangle(imgVideo, (x1,y1-40), (x2,y1), (0,255,0), -2)
        cv2.putText(imgVideo, identify + ", " + str(round(min_dist, 2)), (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)

    cv2.imshow('Face Recognition', imgVideo)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 70ms/step
yasir , 6.9614954
1/1 [==============================] - 0s 62ms/step
yasir , 6.1767716
1/1 [==============================] - 0s 60ms/step
yasir , 6.8225856
1/1 [==============================] - 0s 63ms/step
yasir , 5.7483864
1/1 [==============================] - 0s 62ms/step
yasir , 6.2758236
1/1 [==============================] - 0s 64ms/step
yasir , 5.7071295
1/1 [==============================] - 0s 57ms/step
yasir , 6.9941254
1/1 [==============================] - 0s 58ms/step
yasir , 6.1990457
1/1 [==============================] - 0s 59ms/step
yasir , 6.2381806
1/1 [==============================] - 0s 59ms/step
yasir , 5.799041
1/1 [==============================] - 0s 79ms/step
yasir , 6.5435457
1/1 [==============================] - 0s 62ms/step
yasir , 4.321355
1/1 [==============================] - 0s 77ms/step
yasir , 5.7472014
1/1 [==============================] - 0s 77ms/step
yasir , 6.672077
1/1 [==================